# Imports

In [1]:
import csv
import os
from utils import get_id_list, get_data, write_file

from dotenv import load_dotenv
load_dotenv()

True

# Grabbing All Movie IDs

In [2]:
TMBD_API_KEY = os.getenv('TMBD_API_KEY')
YEARS = [2022, 2023]

# Writing all IDs to CSV file

In [3]:
for year in YEARS:
    # Grab list of ids for all films made in {YEAR}
    movie_list = get_id_list(TMBD_API_KEY, year)
    print(movie_list)

    FILE_NAME = f'./data/{year}_movie_collection_data.csv'

    # Creating file
    with open(FILE_NAME, 'w') as f:
        writer = csv.writer(f)

    # Iterate through list of ids to get data
    for id in movie_list:
        text = get_data(TMBD_API_KEY, id)
        write_file(FILE_NAME, text)

['885303', '1028703', '872542', '799155', '76600', '965571', '843380', '315162', '949567', '949697', '951546', '361743', '624091', '610150', '934194', '508947', '414906', '507086', '942012', '505642', '675353', '436270', '438148', '610411', '585083', '818647', '995906', '616037', '851644', '829557', '983507', '985939', '1187998', '829560', '766507', '453395', '585511', '406759', '338953', '916224', '718930', '879957', '779782', '783675', '661374', '1001835', '545611', '1015908', '774825', '966220', '615777', '888001', '836225', '942881', '818893', '856433', '663712', '760104', '988591', '900667', '744276', '833326', '335787', '799379', '619979', '615173', '937895', '864101', '606870', '1020969', '49046', '601796', '854006', '717728', '646385', '536554', '830784', '629542', '876792', '539681', '1017794', '526896', '518896', '1008148', '668482', '736526', '785084', '752623', '457232', '882598', '642885', '338958', '760161', '864231', '698508', '893712', '614934', '705996', '804095', '945

# Random Testing

In [8]:
import wikipediaapi

title = 'Ruthless'

wiki_wiki = wikipediaapi.Wikipedia(
    user_agent='FilmBot (ed.izaguirre@pm.me)',
    language='en',
    extract_format=wikipediaapi.ExtractFormat.WIKI
)

p_wiki = wiki_wiki.page(title)

is_disambiguation = p_wiki.title.endswith("(disambiguation)")
print(is_disambiguation)

print(p_wiki.title)
print(p_wiki.text)

False
Ruthless
Ruthless refers to a lack of conscience or empathy.
Ruthless may also refer to:

Music
Ruthless!, a 1992 musical
Ruthless (Ace Hood album), 2009
Ruthless (Bizzy Bone album), 2008
Ruthless (Gary Allan album), 2021
Ruthless Records, a hip hop record label
Ruthless Records (Chicago), a punk record label

Other uses
Ruthless (1948 film), a 1948 film starring Zachary Scott
Ruthless (2023 film), a 2023 film starring Dermot Mulroney
Ruthless (horse), a racehorse
Ruthless (novel), a Pretty Little Liars novel by Sara Shepard
Ruthless: Scientology, My Son David Miscavige, and Me, a 2016 book by Ron Miscavige and Dan Koon
Ruthless (TV series), TV series premiered 2020, by Tyler Perry

See also
Ruthless Stakes, a horse race run at Aqueduct Racetrack
Tom Clancy's ruthless.com, a computer game
